### GPU 초기화

In [1]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

### 라이브러리  import

In [2]:
import argparse

from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer

from trl import (
    ModelConfig,
    ScriptArguments,
    SFTConfig,
    SFTTrainer,
    TrlParser,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config
)

c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 훈련 Argument 정의

In [3]:
model_args = ModelConfig(
    model_name_or_path='../models/EXAONE-3.5-2.4B-Instruct/',
    torch_dtype="bfloat16",
    trust_remote_code=True,
    
    
    ####################
    # lora 
    ####################
    use_peft= True,
    lora_r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    lora_target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head"
    ],
    lora_task_type="CAUSAL_LM",
    
    ####################
    # quantization
    ####################
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    use_bnb_nested_quant= False    
)

In [4]:
training_args = SFTConfig(
    output_dir = '../models/EXAONE-3.5-2.4B-Instruct-SFT/',
    eval_strategy='no',
    push_to_hub=False,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=300,
    # max_steps=,
    fp16=True,
    packing = False,
    max_seq_length = 1024,
    # packing=True
)

In [5]:
script_args = ScriptArguments(
    dataset_name='../datasets/train_data/train_data_v2',
    dataset_config = None,
    dataset_train_split='train'
)

### 훈련 함수

In [6]:
def train_llm(script_args, training_args, model_args):
    quantization_config = get_quantization_config(model_args)
    
    model_kwargs = dict(
        revision = model_args.model_revision,
        trust_remote_code = model_args.trust_remote_code,
        attn_implementation = model_args.attn_implementation,
        torch_dtype = model_args.torch_dtype,
        use_cache=False if training_args.gradient_checkpointing else True,
        device_map = get_kbit_device_map() if quantization_config is not None else None,
        quantization_config = quantization_config,
    )
    
    training_args.model_init_kwargs = model_kwargs
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path, trust_remote_code=model_args.trust_remote_code, use_fast=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    dataset = load_from_disk(script_args.dataset_name)
    # formatting_func 예시
    def formatting_prompt_func(example):
        output_texts = []
        system_message = "너는 병역이행법 상담을 도와주는 챗봇이야."
        for i in range(len(example['question'])):
            text = f"[|system|]{system_message}[|endofturn|]\n[|user|]{example['question'][i]}[|endofturn|]\n[|assistant|]{example['answer'][i]}[|endofturn|]"
            output_texts.append(text)
        return output_texts
    trainer = SFTTrainer(
        model = model_args.model_name_or_path,
        args = training_args,
        train_dataset = dataset[script_args.dataset_train_split],
        processing_class = tokenizer,
        peft_config = get_peft_config(model_args),
        formatting_func = formatting_prompt_func,
    )
    
    trainer.train()
    
    trainer.save_model(training_args.output_dir)
    if training_args.push_to_hub:
        trainer.push_to_hub(dataset_name=script_args.dataset_name)

### 훈련

In [7]:
train_llm(script_args, training_args, model_args)



Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.25s/it]
c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\tuners\tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(
  1%|          | 2/300 [06:21<14:21:48, 173.52s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  1%|▏         | 4/300 [13:09<14:34:41, 177.30s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Se

{'loss': 34.1785, 'grad_norm': 9.935717582702637, 'learning_rate': 0.00019945218953682734, 'epoch': 5.0}


  4%|▍         | 12/300 [40:46<14:51:40, 185.77s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  5%|▍         | 14/300 [47:56<15:00:34, 188.93s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  5%|▌         | 16/300 [55:00<15:04:33, 191.11s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` 

{'loss': 20.7715, 'grad_norm': 7.12577486038208, 'learning_rate': 0.00019781476007338058, 'epoch': 10.0}


  7%|▋         | 22/300 [1:15:43<14:34:54, 188.83s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  8%|▊         | 24/300 [1:22:45<14:29:59, 189.13s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  9%|▊         | 26/300 [1:30:09<14:42:55, 193.34s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_la

...

{'loss': 0.6836, 'grad_norm': 5.980234146118164, 'learning_rate': 5.478104631726711e-07, 'epoch': 145.0}


 97%|█████████▋| 292/300 [17:17:40<23:44, 178.03s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
 98%|█████████▊| 294/300 [17:24:37<18:11, 181.98s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
 99%|█████████▊| 296/300 [17:31:03<11:52, 178.20s/it]c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layer

{'loss': 0.6815, 'grad_norm': 5.510958194732666, 'learning_rate': 0.0, 'epoch': 150.0}


c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
100%|██████████| 300/300 [17:45:22<00:00, 213.07s/it]
c:\ProgramData\Anaconda3\envs\env_mark2\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


{'train_runtime': 63922.2707, 'train_samples_per_second': 1.535, 'train_steps_per_second': 0.005, 'train_loss': 4.722296509742737, 'epoch': 150.0}
